In [1]:
import csv
import pandas as pd
import numpy as np
import pickle

In [2]:
import nltk
import string
#Importing the stopwords
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
#Initializing the WordNetLemmatizer
lemmer = nltk.stem.WordNetLemmatizer()
#A dictionary reference for replacing special characters
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemTokens(tokens):
       return [lemmer.lemmatize(token,'v') for token in tokens if token not in set(stopwords.words('english'))]
def LemNormalize(text):
       return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))
# tokenized_dataset.drop_duplicates(subset ="utterance",keep = "first", inplace = True)
# tokenized_dataset['tokenized_text']=tokenized_dataset.utterance.apply(lambda row: LemNormalize(row))
def LemAllTokens(tokens):
       return [lemmer.lemmatize(token,'v') for token in tokens]
def LemAllNormalize(text):
       return LemAllTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [4]:
def generate_response(user_input):
    tennisrobo_response = ''
    article_sentences=prompts.copy()
    article_sentences.append(user_input)

    word_vectorizer = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words('english'))
    all_word_vectors = word_vectorizer.fit_transform(article_sentences)
    similar_prompt_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
    similar_prompt_number = similar_prompt_vector_values.argsort()[0][-2]

    matched_vector = similar_prompt_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched != 0 :
        utterances = dic[article_sentences[similar_prompt_number]].copy()
        utterances.append(user_input)
        word_vectorizer = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words('english'))
        all_word_vectors = word_vectorizer.fit_transform(utterances)
        similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
        similar_sentence_number = similar_vector_values.argsort()[0][-2]
        i= -2
        while(similar_sentence_number==len(utterances)-2):
            i-=1
            similar_prompt_number = similar_prompt_vector_values.argsort()[0][-2]
            utterances = dic[article_sentences[similar_prompt_number]].copy()
            utterances.append(user_input)
            word_vectorizer = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words('english'))
            all_word_vectors = word_vectorizer.fit_transform(utterances)
            similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
            similar_sentence_number = similar_vector_values.argsort()[0][-2]
        tennisrobo_response = tennisrobo_response + utterances[similar_sentence_number+1]
        print("prompt : ",article_sentences[similar_prompt_number])
        print("original  : ",utterances[similar_sentence_number])
        return tennisrobo_response
    else:
        tennisrobo_response = tennisrobo_response + "I don't know what to say, but I undertand that you are sad"
        return tennisrobo_response

In [5]:
def getReady():
    loaded_model = pickle.load(open('SentimentModel.sav', 'rb'))
    Countvectorizer=pickle.load(open('Countvectorizer.pickle', 'rb'))
    indeces_to_delete=pickle.load(open('indeces_to_delete.pickle', 'rb'))

In [6]:
def getSentiment(sentence):
    loaded_model = pickle.load(open('SentimentModel.sav', 'rb'))
    Countvectorizer=pickle.load(open('Countvectorizer.pickle', 'rb'))
    indeces_to_delete=pickle.load(open('indeces_to_delete.pickle', 'rb'))
    getReady()
    X_new = Countvectorizer.transform(sentence)
    X_new_simple = np.delete(X_new.toarray(), indeces_to_delete, axis=1)
    return loaded_model.predict(X_new_simple)[0]

In [7]:
getSentiment(['I have cancer'])

'sad'

In [11]:
def generate_response(user_input,name):
    word_vectorizer = pickle.load(open(name+'_word_vectorizer.sav', 'rb'))
    all_word_vectors=pickle.load(open(name+'_all_word_vectors.pickle', 'rb'))
    utterances=pickle.load(open(name+'_utterance.pickle', 'rb'))
    prompts_sentences=pickle.load(open(name+'_prompts_sentences.pickle', 'rb'))
    tennisrobo_response = ''
    user_input_vector= word_vectorizer.transform([user_input])
    similar_prompt_vector_values = cosine_similarity(user_input_vector[0], all_word_vectors)
    similar_sentence_number = similar_prompt_vector_values.argsort()[0][-1]
                                      
    matched_vector = similar_prompt_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-1]
    if vector_matched != 0 and similar_sentence_number< len(utterances)-1:
        first = '' + utterances[similar_sentence_number+1]
        similar_sentence_second_number = similar_prompt_vector_values.argsort()[0][-2]
        second = '' + utterances[similar_sentence_second_number+1]
        similar_sentence_third_number = similar_prompt_vector_values.argsort()[0][-3]
        third = '' + utterances[similar_sentence_third_number+1]
#         print("first : ",first)
#         print(matched_vector[-1])
#         print("second : ",second)
#         print(matched_vector[-2])
#         print("third : ",third)
#         print(matched_vector[-3])
        Allreplies=[user_input,first,second,third]
        Allprompts=[user_input,prompts_sentences[similar_sentence_number],prompts_sentences[similar_sentence_second_number],prompts_sentences[similar_sentence_third_number]]
        Countvectorizer = CountVectorizer(tokenizer = LemAllNormalize, ngram_range=(1,1))
        counts_vector= Countvectorizer.fit_transform(Allprompts)
        count_prompt_cosine_similarity = cosine_similarity(counts_vector[0], counts_vector)
#         print(counts_vector[0])
        count_matched_vector = count_prompt_cosine_similarity.flatten()
        count_matched_vector.sort()
        fi=count_prompt_cosine_similarity.argsort()[0][0]
#         print(Allprompts[fi],Allreplies[fi])
#         print(count_matched_vector[0])
        si=count_prompt_cosine_similarity.argsort()[0][1]
#         print(Allprompts[si],Allreplies[si])
#         print(count_matched_vector[1])
        ti=count_prompt_cosine_similarity.argsort()[0][2]
#         print(Allprompts[ti],Allreplies[ti])
#         print(count_matched_vector[2])
        foi=count_prompt_cosine_similarity.argsort()[0][3]
#         print(Allprompts[foi],Allreplies[foi])
#         print(count_matched_vector[3])
        return Allreplies[ti]
    else:
        return "I don't know what to say, but I undertand that you are"+name

In [12]:
import random
greeting_inputs = ("hey","I am good", "good morning", "good evening","hello", "morning", "evening", "hi", "whatsup")
greeting_responses = ["hey","You seems happy? Is that true?", "hey hows you?", "*nods*", "hello, how you doing", "hello", "Welcome, I am good and you?"]
goodbyes_inputs = ('thanks','thank you very much', 'thank you', "bye","goodbye","no")
goodbyes_responses = ["Most welcome",'thanks','thank you very much', 'thank you', "bye","goodbye"]

def generate_greeting_response(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)
def generate_goodbyes_response(greeting):
    for token in greeting.split():
        if token.lower() in goodbyes_inputs:
            return random.choice(goodbyes_responses)
def wrong_sentiment(greeting):
    for token in greeting.split():
        if token.lower() =="no":
            return True

In [13]:
continue_dialogue = True
sentiment=None
print("Venti: Hello, I am your friend Venti. Do you wanna talk a little with me? :")
while(continue_dialogue == True):
    
    human_text = input()
    human_text = human_text.lower()
    if human_text != 'bye':
        if generate_goodbyes_response(human_text) != None:
            continue_dialogue = False
            print("Venti: " + generate_goodbyes_response(human_text))
        else:
            if sentiment == None:
                sentiment= getSentiment([human_text])
                print("Venti: You seems", sentiment, "? Is that true?")
                human_text = input()
                human_text = human_text.lower()
                print("Venti: Ok, Do you wanna talk more about it?")
                human_text = input()
                human_text = human_text.lower()
                if wrong_sentiment(human_text)==True:
                    sentiment= getSentiment([human_text])
#                 print("typing...")
                print("Venti: ", generate_response(human_text,sentiment))
            else:
#                 print("typing...")
                print("Venti: ", generate_response(human_text,sentiment))
    else:
        continue_dialogue = False
        print("Venti: Good bye and take care of yourself...")


Venti: Hello, I am your friend Venti. Do you wanna talk a little with me? :
yes
Venti: You seems surprised ? Is that true?
no, tell me about yourself
Venti: Ok, Do you wanna talk more about it?
yes 


C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'didnt', 'doesnt', 'dont', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldve', 'thatll', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Venti:  I like to do that for people when they don't expect it or when they need it most


KeyboardInterrupt: Interrupted by user

In [14]:
"""Translates text into the target language.

Target must be an ISO 639-1 language code.
See https://g.co/cloud/translate/v2/translate-reference#supported_languages
"""
import six
from google.cloud import translate_v2 as translate
def translate_text(target, text):


    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

    print(u"Text: {}".format(result["input"]))
    print(u"Translation: {}".format(result["translatedText"]))
    print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))

translate_text('en', "اهلا")

ModuleNotFoundError: No module named 'google'